# Imports

In [1]:
from tqdm.notebook import tqdm_notebook
import time
import tmdbsimple as tmdb
import pandas as pd
import numpy as np

In [8]:
import os
FOLDER = "Data/MovieProject/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'akas.csv.gz',
 'basics.csv.gz',
 'Data',
 'ratings.csv.gz']

In [3]:
import json
with open('C:\\Users\\javaughn\\.secret\\tmdb_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

# API - Key

In [4]:
tmdb.API_KEY = login['api-key']

# Functions

In [5]:
def get_movie_with_rating(movie_id):
    # Get movie and release dates
    movie = tmdb.Movies(movie_id)
    # Construct output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

In [6]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [7]:
YEARS_TO_GET = [2000, 2001]

In [9]:
basics = pd.read_csv("Data//MovieProject//basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


# Extract years 2000 and 2001

In [11]:
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    #Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie ids to pandas Series
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    #Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            write_json({'budget': 0, 'revenue': 0, 'imdb_id': movie_id,
                        'certification': 'None'}, JSON_FILE)
            continue
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False) 

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1403 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1517 [00:00<?, ?it/s]

# Combine years 2000 and 2001

In [15]:
df_2000 = pd.read_csv("Data/MovieProject/final_tmdb_data_2000.csv.gz", low_memory = False)

In [16]:
df_2001 = pd.read_csv("Data/MovieProject/final_tmdb_data_2001.csv.gz", low_memory = False)

In [17]:
df_2000_2001 = pd.concat([df_2000, df_2001])

In [18]:
df_2000_2001.to_csv("Data/MovieProject/tmdb_2000_2001_results_combined.csv.gz", compression = "gzip", index = False)